### Setting paths and imports

In [1]:
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
module_path

os.chdir(module_path)

# from src.evaluation.evaluator_manager import EvaluatorManager
# from src.evaluation.future.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.utils.context import Context
from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

from src.evaluation.future.evaluator_manager_triplets import EvaluatorManager

config_f_name = 'generate_minimize/tcr-ablation-cycles/dce/dce-lcls-20/generate_minimize0.jsonc'

### Evaluating the explainer

In [2]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)
eval_manager.evaluate()

Generating context for: c:\Users\rodri\OneDrive\Documents\Projects\GRETEL stuff\GRETEL\lab\config\generate_minimize/tcr-ablation-nodes/dce-random/dce-random-128/generate_minimize0.jsonc
2025-01-14 23:08:57,1747350114 | INFO | 16392 - Executing: c:\Users\rodri\OneDrive\Documents\Projects\GRETEL stuff\GRETEL\lab\config\generate_minimize/tcr-ablation-nodes/dce-random/dce-random-128/generate_minimize0.jsonc Run: 1
2025-01-14 23:08:57,1747350127 | INFO | 16392 - Creating the evaluation manager....................................
2025-01-14 23:08:57,1747350143 | INFO | 16392 - Creating the evaluators......................................................
2025-01-14 23:09:00,1747353145 | INFO | 16392 - Loading: TreeCyclesRand-178a56d73f41c72ff44bff2bf4defcab
2025-01-14 23:09:02,1747354970 | INFO | 16392 - Created: TreeCyclesRand-178a56d73f41c72ff44bff2bf4defcab
2025-01-14 23:09:05,1747357543 | INFO | 16392 - Loading: TreeCyclesOracle-1855fcd3cf5fe16028f5dde6d79da96b
2025-01-14 23:09:06,1747358

In [3]:
exp_inst = []
for exp in eval_manager.evaluators[0]._explanations:
    exp.input_instance._dataset = None
    exp.counterfactual_instances[0]._dataset = None
    
exp_inst = [(exp.input_instance, exp.counterfactual_instances[0]) for exp in eval_manager.evaluators[0]._explanations]

In [4]:
import os
import pickle

pickle_path = os.path.join(module_path, 'lab', 'pickles', 'union.pkl')
pickle_dir = os.path.dirname(pickle_path)

if not os.path.exists(pickle_dir):
    os.makedirs(pickle_dir)

with open(pickle_path, 'wb') as pickle_file:
    pickle.dump(exp_inst, pickle_file)

In [5]:
with open(os.path.join(module_path, 'lab', 'pickles', 'union.pkl'), 'rb') as clear_file:
    clear_exp = pickle.load(clear_file)

### Aggregating the stats

In [6]:
results_path = os.path.join(module_path, 'lab', 'output', 'results')
stats_file_path = os.path.join(module_path, 'lab', 'stats', 'results.csv')
res = data_analyzer.create_aggregated_dataframe(results_path)
res.to_csv(stats_file_path)
res

,scope,dataset,oracle,explainer,Runtime,Runtime-std,GraphEditDistance,GraphEditDistance-std,FeatureEditDistance,FeatureEditDistance-std,Correctness,Correctness-std,OracleCalls,OracleCalls-std,OracleAccuracy,OracleAccuracy-std,Sparsity,Sparsity-std
0,ablation-nodes,TreeCyclesRand-178a56d73f41c72ff44bff2bf4defcab,TreeCyclesOracle-1855fcd3cf5fe16028f5dde6d79da96b,GenerateMinimize(DCESExplainer-LocalSearch),5.803932,0.000000,2.200000,0.000000,0.000000,0.000000,1.000,0.0,1718.800000,0.000000,1.000000,0.000000,0.011406,0.000000
1,ablation-random-nodes-128,TreeCyclesRand-178a56d73f41c72ff44bff2bf4defcab,TreeCyclesOracle-1855fcd3cf5fe16028f5dde6d79da96b,GenerateMinimize(DCESExplainer-Random),12.563278,0.000000,2.200000,0.000000,0.000000,0.000000,1.000,0.0,4306.000000,0.000000,1.000000,0.000000,0.011406,0.000000
2,meta,ASD-15273954d84e872cf0b021cd4477bfdc,ASDOracle-9e4f3586dc330143b7849fc540b25739,GenerateMinimize(DCESExplainer-Dummy),0.153786,0.000000,1029.727273,0.000000,0.000000,0.000000,1.000,0.0,104.000000,0.000000,0.727273,0.000000,2.315111,0.000000
3,meta,ASD-15273954d84e872cf0b021cd4477bfdc,ASDOracle-9e4f3586dc330143b7849fc540b25739,GenerateMinimize(DCM-Dummy),23.539063,0.000000,6544.000000,0.000000,0.000000,0.000000,1.000,0.0,3.000000,0.000000,0.727273,0.000000,14.714456,0.000000
4,meta,BBBP-c0cc58bdf4acf93936322304bf6b669e,OracleTorch-67fdf45735a1c8aaece5ab786ca3c38e,GenerateMinimize(DCESExplainer-LocalSearch),26.122186,0.000000,2.431373,0.000000,33.526114,0.000000,1.000,0.0,7052.808824,0.000000,0.852941,0.000000,0.016821,0.000000
5,meta,BBBP-c0cc58bdf4acf93936322304bf6b669e,OracleTorch-67fdf45735a1c8aaece5ab786ca3c38e,GenerateMinimize(DCESExplainer-LocalSearchMult...,22.593451,0.000000,3.632353,0.000000,41.866307,0.000000,1.000,0.0,6689.549020,0.000000,0.906863,0.000000,0.025100,0.000000
6,meta,BBBP-c0cc58bdf4acf93936322304bf6b669e,OracleTorch-67fdf45735a1c8aaece5ab786ca3c38e,GenerateMinimize(DCESExplainer-Random),10.564593,0.109784,10.450980,0.058824,128.746100,3.061536,1.000,0.0,6037.196078,9.803922,0.879902,0.026961,0.069879,0.000697
7,meta,IMDB-BINARY-3fcfb2d32433841064a43599dd908a5b,OracleTorch-cdb47ad283f8190a0dece30559f3a9be,GenerateMinimize(Cascade-Dummy),1.963296,0.000000,75.770000,0.000000,506.066234,0.000000,1.000,0.0,813.280000,0.000000,0.720000,0.000000,0.397100,0.000000
8,meta,Synthie-2703e12cc033a6e1b64e835e11da25e3,OracleTorch-5449ec726ac893cfdaed806068e27c41,GenerateMinimize(DCESExplainer-Dummy),1.295360,0.000000,197.325000,0.000000,741.664739,0.000000,1.000,0.0,403.000000,0.000000,1.000000,0.000000,1.048900,0.000000
9,meta,Synthie-2703e12cc033a6e1b64e835e11da25e3,OracleTorch-5449ec726ac893cfdaed806068e27c41,GenerateMinimize(DCESExplainer-LocalSearch),21.539155,0.000000,3.700000,0.000000,577.922850,0.000000,1.000,0.0,4958.725000,0.000000,1.000000,0.000000,0.020097,0.000000


### Visualizing individual instances

In [7]:
evaluator = eval_manager.evaluators[0]
evaluator

In [8]:
exp = evaluator.explanations[2]
og_inst = exp.input_instance
cf_inst = exp.counterfactual_instances[0]

# Drawing Instances

In [9]:
import pickle

with open(os.path.join(module_path, 'lab', 'pickles', 'cf2.pkl'), 'rb') as cf2_file:
    cf2_exp = pickle.load(cf2_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'clear.pkl'), 'rb') as clear_file:
    clear_exp = pickle.load(clear_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'irand.pkl'), 'rb') as irand_file:
    irand_exp = pickle.load(irand_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'obs.pkl'), 'rb') as obs_file:
    obs_exp = pickle.load(obs_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'rsgg.pkl'), 'rb') as rsgg_file:
    rsgg_exp = pickle.load(rsgg_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'bidirectional.pkl'), 'rb') as bi_file:
    bi_exp = pickle.load(bi_file)

with open(os.path.join(module_path, 'lab', 'pickles', 'union.pkl'), 'rb') as union_file:
    union_exp = pickle.load(union_file)


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\rodri\\OneDrive\\Documents\\Projects\\GRETEL stuff\\GRETEL\\lab\\pickles\\cf2.pkl'

In [ ]:
i = 7

og_inst = cf2_exp[i][0]
cf2_cf = cf2_exp[i][1]
print(og_inst.id, cf2_cf.id)

clear_cf = clear_exp[i][1]
print(clear_exp[i][0].id, clear_cf.id)

irand_cf = irand_exp[i][1]
print(irand_exp[i][0].id, irand_cf.id)

obs_cf = obs_exp[i][1]
print(obs_exp[i][0].id, obs_cf.id)

rsgg_cf = rsgg_exp[i][1]
print(rsgg_exp[i][0].id, rsgg_cf.id)

union_cf = union_exp[i][1]
print(union_exp[i][0].id, union_cf.id)

bi_cf = bi_exp[i][1]
print(bi_exp[i][0].id, bi_cf.id)

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf2_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, clear_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, irand_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, obs_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, rsgg_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, union_cf, position=pos)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, bi_cf, position=pos)

# End Paper Experiments

In [ ]:

changes = data_analyzer.get_cf_changes(og_inst, cf_inst, False)

In [ ]:
common_edges = changes['common edges']
added_edges = changes['added edges']
removed_edges = changes['removed edges']
common_nodes = changes['common nodes']
added_nodes = changes['added nodes']
removed_nodes = changes['removed nodes']

print(f'added edges: {added_edges}')
print(f'removed_edges: {removed_edges}')
print(f'added nodes: {added_nodes}')
print(f'removed nodes: {removed_nodes}')

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf_inst, position=pos)